In [108]:
import numpy as np
import plotly.express as px
from keras import Input, Sequential
from keras.src.layers import Dense
from keras.src.optimizers import Adam
import plotly.graph_objects as go
from keras.src.saving.saving_api import load_model

In [71]:
series = np.sin(0.1 * np.arange(200)) - np.random.randn(200) * 0.1

px.line(series)

In [88]:
T = 20
X = []
Y = []

for t in range(len(series) - T):
    x = series[t: t + T]
    y = series[t + T]
    X.append(x)
    Y.append(y)

X = np.array(X).reshape(-1, T)
Y = np.array(Y)
print(f"X.shape: {X.shape}\nY.shape: {Y.shape}")
N = len(X)

X.shape: (180, 20)
Y.shape: (180,)


In [94]:
model = Sequential()
model.add(Input(shape=(T,)))
model.add(Dense(units=4, activation="relu"))
model.add(Dense(units=1))

model.compile(loss="mse", optimizer=Adam(learning_rate=0.01))

history = model.fit(x=X[:-N // 2], y=Y[:-N // 2], epochs=150, validation_data=(X[-N // 2:], Y[-N // 2:]))

Epoch 1/150
3/3 [==============================] - 1s 126ms/step - loss: 0.5885 - val_loss: 0.4632
Epoch 2/150
3/3 [==============================] - 0s 33ms/step - loss: 0.3330 - val_loss: 0.3111
Epoch 3/150
3/3 [==============================] - 0s 33ms/step - loss: 0.2454 - val_loss: 0.2714
Epoch 4/150
3/3 [==============================] - 0s 35ms/step - loss: 0.2562 - val_loss: 0.2512
Epoch 5/150
3/3 [==============================] - 0s 36ms/step - loss: 0.2370 - val_loss: 0.2341
Epoch 6/150
3/3 [==============================] - 0s 35ms/step - loss: 0.2082 - val_loss: 0.2107
Epoch 7/150
3/3 [==============================] - 0s 37ms/step - loss: 0.1669 - val_loss: 0.1975
Epoch 8/150
3/3 [==============================] - 0s 31ms/step - loss: 0.1425 - val_loss: 0.1826
Epoch 9/150
3/3 [==============================] - 0s 33ms/step - loss: 0.1272 - val_loss: 0.1539
Epoch 10/150
3/3 [==============================] - 0s 32ms/step - loss: 0.1038 - val_loss: 0.1181
Epoch 11/150
3/3 [

In [95]:
def draw_loss(history):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history['loss'], name="loss"))
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_loss'], name="val_loss"))
    fig.show()


draw_loss(history)

In [23]:
# # Wrong predictions
# validation_targets = Y[-N // 2:]
# validation_predictions = []
# 
# i = -N // 2
# 
# while len(validation_predictions) < len(validation_targets):
#     p = model.predict(X[i].reshape(1, -1), verbose=0)[0, 0]
#     i += 1
# 
#     validation_predictions.append(p)

In [98]:
# Correct predictions
validation_targets = Y[-N // 2:]
validation_predictions = []

last_x = X[-N // 2]

while len(validation_predictions) < 500:
    p = model.predict(last_x.reshape(1, -1), verbose=0)[0, 0]

    validation_predictions.append(p)

    last_x = np.roll(last_x, -1)
    last_x[-1] = p

In [99]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=validation_targets, name="targets"))
fig.add_trace(go.Scatter(y=validation_predictions, name="predictions"))
fig.show()

In [112]:
m = load_model("best_model.keras")

preds = []

last_x = X[0]

while len(preds) < 180:
    p = m.predict(last_x.reshape(1, -1))[0, 0]

    preds.append(p)

    last_x = np.roll(last_x, -1)
    last_x[-1] = p

1/1 [==============================] - 0s 30ms/step


In [113]:
fig = go.Figure()
fig.add_scatter(y=Y, mode="lines", name="True")
fig.add_scatter(y=preds, mode="lines", name="Predicts")
fig.show()